# Calculating momentum based on 12-month lookback period
> To calculate the cumulative return over the past 12 months, we take the net return streams from each month and turn them into gross returns by adding 1. Thus, if Apple's net returns for January are –10.77 percent, Apple's gross returns for January are 0.8923 (–0.1077 + 1). Then, we multiply all the gross return series (i.e., months) and subtract 1 to find the cumulative 12-month net return. ([Location 2467](https://readwise.io/to_kindle?action=open&asin=B01LY6P2LB&location=2467))

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import arrow
import pandas as pd
import numpy as np

import dataclasses
import math
import sys
sys.path.append('../src')

In [ ]:
import quality_momentum as qm

In [ ]:
import importlib; importlib.reload(qm)

In [ ]:
metric = qm.calculate_momentum.get_monthly_momentum('BA', arrow.get('2017-12-03'))

In [ ]:
metric

In [ ]:
df.to_csv('./ticker_quandl_data.csv')

In [ ]:
monthly_returns = df["daily_returns"].resample("M").apply(lambda x: ((x + 1).cumprod()).last("D"))
cumulative_return = np.prod(monthly_returns) - 1

In [ ]:
print(cumulative_return)

In [ ]:
qm.calculate_momentum.calculate_fips_number(df, -.456)

In [ ]:
now = arrow.get('2017-12-03')
equities = qm.calculate_momentum.get_universe_of_equities(now)
momentum_measures = [qm.calculate_momentum.get_monthly_momentum(e, now) for e in equities]
df = pd.DataFrame.from_records([dataclasses.asdict(x) for x in momentum_measures], index='ticker')
df['quantile_rank'] = pd.qcut(df['momentum'], 10, labels=False)

In [ ]:
top_decile_momentum_equities = df[df['quantile_rank'] == 9]
top_quality_momentum = pd.qcut(top_decile_momentum_equities['fip'], 2, labels=['high_quality', 'low_quality'])
top_decile_momentum_equities = top_decile_momentum_equities.assign(quality_momentum=top_quality_momentum.values)
# equities_to_buy = top_decile_momentum_equities[top_decile_momentum_equities['quality_momentum'] == 1]
equities_to_buy = top_decile_momentum_equities[top_decile_momentum_equities['quality_momentum'] == 'high_quality']

In [ ]:
print(equities_to_buy)

In [ ]:
now = arrow.get('2017-12-03')
tickers = qm.calculate_momentum.get_quality_momentum_stocks(now, 2)

In [ ]:
tickers = equities_to_buy
available_cash = 100000

In [ ]:
data = {x: available_cash / len(tickers) for x in tickers}
df = pd.DataFrame.from_dict(data, orient="index")

In [ ]:
df.index

In [ ]:
df.head(10)

In [ ]:
portfolio = qm.portfolio.purchase_new_shares(arrow.get('2017-12-03'), 100000, 4, qm.portfolio.WeightType.equal_weighted)

In [ ]:
print(portfolio.head())

In [ ]:
now = arrow.get('2017-12-03')
import pandas_datareader as pdr

equities = {}
for equity in portfolio.index.tolist():
    # example from https://nbviewer.ipython.org/github/twiecki/financial-analysis-python-tutorial/blob/master/1.%20Pandas%20Basics.ipynb
    adj_close_series = pdr.quandl.QuandlReader(equity, now.shift(days=-2).format('YYYY-MM-DD'), now.format('YYYY-MM-DD')).read()['AdjClose']
    # grab first adjusted close value available
    equities[equity] = {"adj_close": adj_close_series[0]}
df = pd.DataFrame.from_dict(data=equities, orient='index')

In [ ]:
df

In [ ]:
portfolio = portfolio.join(df)

In [ ]:
portfolio

In [ ]:
portfolio['num_shares_to_purchase'] = (portfolio['position_size'] / portfolio['adj_close']).apply(np.floor)

In [ ]:
portfolio